In [1]:
from imp import reload
import cProfile, time

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import cca_pytorch, cca, gp

import torch

In [6]:
T = 20
feats = 50
d = 5
K = gp.gen_kernel('squared_exp', temporal_scale=3., spatial_scale=.5)
Xt, _, _, _ = gp.embed_gp(1000, feats, d, K, np.eye(feats), T, num_to_concat=100)
CovXt = cca.calc_cross_cov_mats_from_data(Xt, 2 * T)
V_init = scipy.stats.ortho_group.rvs(feats)[:, :d]

In [7]:
t = time.time()
vp = cca_pytorch.run_cca(CovXt, d, verbose=False, init=V_init, dtype=torch.float32)
print(time.time() - t)
print(cca_pytorch.build_loss(CovXt, d, dtype=torch.float32)(vp))

15.019512414932251
tensor(-2.1349, device='cuda:0')


In [8]:
t = time.time()
va = cca.run_cca(CovXt, d, verbose=False, init=V_init)
print(time.time() - t)
print(cca.build_loss(CovXt, d)(va))

<class 'numpy.ndarray'>
41.919169664382935
-2.134852625006482


In [5]:
np.linalg.eigvalsh(va.T.dot(vp))

array([-1.42824913, -0.88047061, -0.51274582,  0.35846769,  1.34995789])